# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


👉 There are fewer rows in the new CSV file because the initial files contains empty rows.

# Part II. Data Modeling with Apache Cassandra

### The denormalized data, stored in 'event_datafile_new.csv', is presented as follows:

In [5]:
data = pd.read_csv('event_datafile_new.csv')
data.head(10)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Rokia TraorÃÂ©,Stefany,F,0,White,274.88608,free,"Lubbock, TX",693,Zen,83
1,Camila,Tucker,M,1,Garrison,230.81751,free,"Oxnard-Thousand Oaks-Ventura, CA",555,Abrazame (Version Acustica),40
2,Carl Thomas,Tucker,M,0,Garrison,196.67546,free,"Oxnard-Thousand Oaks-Ventura, CA",698,You Ain't Right (Album Version),40
3,N.E.R.D.,James,M,0,Martin,242.99057,free,"Dallas-Fort Worth-Arlington, TX",78,Provider (Remix Radio Edit),79
4,Lil Jon / The East Side Boyz / DJ Flexx,Jacqueline,F,3,Lynch,285.30893,paid,"Atlanta-Sandy Springs-Roswell, GA",589,Aww Skeet Skeet,29
5,Enya,Jacqueline,F,4,Lynch,175.85587,paid,"Atlanta-Sandy Springs-Roswell, GA",589,The Sun In The Stream,29
6,Furthermore,Jacqueline,F,5,Lynch,278.41261,paid,"Atlanta-Sandy Springs-Roswell, GA",589,We Need To Talk (She And I Album Version),29
7,Kid Cudi / MGMT / Ratatat,Jacqueline,F,6,Lynch,295.67955,paid,"Atlanta-Sandy Springs-Roswell, GA",589,Pursuit Of Happiness (nightmare),29
8,Crosby_ Stills & Nash,Jacqueline,F,7,Lynch,186.90567,paid,"Atlanta-Sandy Springs-Roswell, GA",589,Haven't We Lost Enough? (LP Version),29
9,Passion Pit,Jacqueline,F,8,Lynch,174.75873,paid,"Atlanta-Sandy Springs-Roswell, GA",589,Sleepyhead,29


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


👉 None of the columns have NaN values; no issues with the datatype.

👉 For the subsequent data modeling process, the following data type conversions are needed:
* convert 'length' to a float,
* change 'itemInSession', 'sessionId', and 'userId' to integers,
* ensure all other columns are set as text.

In [7]:
data.nunique()

artist           3148
firstName          84
gender              2
itemInSession     123
lastName           86
length           3994
level               2
location           63
sessionId         776
song             5190
userId             96
dtype: int64

👉 None of the columns contain only unique values, as expected.

## Create a connection to Apache Cassandra and create a Sparkify database

#### Creating a Cluster

In [8]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1']) # '127.0.0.1' if you have a locally installed Apache Cassandra instance 

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [9]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}""")
    
except Exception as e:
    print(e)

#### Set Keyspace

In [10]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

## Modeling the database tables

#### Question 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4.
* Table 1: session_history
* Primary key (composite): `session_id` and `item_in_session`. In the query, we will use the WHERE statement to filter data based on certain fields. Additionally, grouping by this combination of fields will ensure that each row is unique.
* Other columns needed: `artist_name`, `song_title`, `song_length`.

#### Question 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.
* Table 2: user_history
* Primary key (composite):`user_id`, `session_id`, `item_in_session`.
    * In the query, we will use the WHERE statement to filter data based on the `user_id` and `session_id` fields. Additionally, grouping by this combination of fields will ensure that each row is unique. Thus, these columns are needed to be used as *partition keys*. 
    * `item_in_session` is needed as a *clustering* column. You can not try to access a column or a clustering column if you have not used the other defined clustering column. That's why it needs to be included into the Primary key.
    * Since `item_in_session` is not a partition key, it's just a clustering column, we need to denote a compound partition key by enclosing `user_id` and `session_id` in parenthasis
* Other columns needed: `artist_name`, `song_title`, `first_name`, `last_name`.

#### Question 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
* Table 3: songs_users_history
* Primary key (composite): `song_title` and `user_id`. This gives us a unique combination of all the songs listend by every user. In the query, we will use the WHERE statement to filter data based on these fields. 
* Other columns needed: `first_name`, `last_name`.

## Create the tables, insert the data and read the data

### Table 1: session_history

In [11]:
# Create the table

query = "CREATE TABLE IF NOT EXISTS session_history"
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length double, \
                    PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# Insert the data into the table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_history (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))

In [13]:
# Select the data from the table to verify that the data model works correctly

# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during 
## sessionId = 338, and itemInSession = 4
query = "SELECT * FROM session_history WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    # Convert rows to a list
    data = list(map(lambda row: [row.session_id, row.item_in_session, row.artist_name, row.song_title, row.song_length], rows))

# Create a dataframe with column names
df1 = pd.DataFrame(data, columns=["session_id", "item_in_session", "artist_name", "song_title", "song_length"])

print(df1.to_string(index=False, justify='left'))

session_id  item_in_session artist_name song_title                        song_length
338         4                Faithless   Music Matters (Mark Knight Dub)  495.3073


### Table 2: users_history

In [14]:
# Create the table

query = "CREATE TABLE IF NOT EXISTS users_history"
query = query + "(user_id int, session_id int, artist_name text, song_title text, \
                    item_in_session int, first_name text, last_name text, \
                    PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [15]:
# Insert the data into the table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:

        query = "INSERT INTO users_history (user_id, session_id, artist_name, song_title, \
                    item_in_session, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[10]), int(line[8]), str(line[0]), str(line[9]), int(line[3]), str(line[1]), str(line[4])))

In [16]:
# Select the data from the table to verify that the data model works correctly

# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "SELECT * FROM users_history WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    # Convert rows to a list
    data = list(map(lambda row: [row.first_name, row.last_name, row.artist_name, row.song_title], rows))

# Create a dataframe with column names
df2 = pd.DataFrame(data, columns=["first_name", "last_name", "artist_name", "song_title"])

print(df2.to_string(index=False, justify='left'))

first_name last_name artist_name        song_title                                        
Sylvie     Cruz       Down To The Bone                                 Keep On Keepin' On
Sylvie     Cruz           Three Drives                                        Greece 2000
Sylvie     Cruz      Sebastien Tellier                                          Kilometer
Sylvie     Cruz          Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...


### Table 3: songs_users_history

In [17]:
# Create the table

query = "CREATE TABLE IF NOT EXISTS songs_users_history"
query = query + "(song_title text, user_id int, first_name text, last_name text, \
                    PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)           

In [18]:
# Insert the data into the table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO songs_users_history (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        session.execute(query, (str(line[9]), int(line[10]), str(line[1]), str(line[4])))

In [19]:
# Select the data from the table to verify that the data model works correctly

# Query 3: Give me every user name (first and last) in my music app history who listened to the song 
##'All Hands Against His Own'
query = "SELECT * FROM songs_users_history WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #print(row.first_name, row.last_name)
    # Convert rows to a list
    data = list(map(lambda row: [row.first_name, row.last_name], rows))

# Create a dataframe with column names
df3 = pd.DataFrame(data, columns=["first_name", "last_name"])

print(df3.to_string(index=False, justify='left'))

first_name  last_name
Jacqueline    Lynch 
     Tegan   Levine 
      Sara  Johnson


### Drop the tables before closing out the sessions

In [20]:
query = "DROP TABLE session_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE users_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE songs_users_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()